In [1]:
import pydicom
from itertools import chain
import os
import numpy
from matplotlib import pyplot, cm
import cv2
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers

Using TensorFlow backend.


# Creation of train Dataset for cropped mass images

In [2]:
# Converts DICOM images in loss less JPEG and save them in a given folder. Possibility of image enhancement by CLAHE algorithm
def dicom_to_jpg(dicom_path, jpg_path, mass_calc, ENHANCE=True):
    if mass_calc=="MASS":
        prefix = "m_"
    elif mass_calc=="CALC":
        prefix = "c_"
    else:
        print("Missing mass_calc argument : set it on 'MASS' if dicom_path contains only \
            mass case, and 'CALC' if it contains only calcification cases ")
        return -1
    clahe1 = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(6,6)) #CLAHE for local contrast enhancing
    clahe2 = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe3 = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(10,10))
    threshold1 = 250*250
    threshold2 = 400*400
    i = 0
    for dirName, subdirList, fileList in os.walk(dicom_path):
        for filename in fileList:
            path = os.path.join(dirName, filename)
            dic = pydicom.read_file(path)
            if (dic.SeriesDescription == "cropped images"):  # check whether the file's segmented image
                i_str = str(i)
                new_name = prefix + (5-len(i_str))*"0"+i_str+".jpg"
                pixels = dic.pixel_array
                size = pixels.size
                if ENHANCE:
                    if size<threshold1:
                        pixels = clahe1.apply(pixels) 
                    elif size<threshold2:
                        pixels = clahe2.apply(pixels)
                    else:
                        pixels = clahe3.apply(pixels)
                cv2.imwrite(os.path.join(jpg_path, new_name), pixels/256., [int(cv2.IMWRITE_JPEG_QUALITY), 100])
                i+=1
    return None

In [3]:
def dataframe_generator(path_file, mass_calc, n_images=None):
    if mass_calc=="MASS":
        prefix = "m_"
    elif mass_calc=="CALC":
        prefix = "c_"
    else:
        print("Missing mass_calc argument : set it on 'MASS' if dicom_path contains only \
            mass case, and 'CALC' if it contains only calcification cases")
        return -1
    full_data = pd.read_csv(path_file, sep=';', header=0)
    full_data.dropna(0, inplace=True, how='all') # Remove fully empty rows
    full_data.reset_index(drop=True, inplace=True)
    descriptions = full_data.pathology.copy()
    if n_images==None:
        n_images = descriptions.shape[0]
    descriptions[descriptions == 'BENIGN_WITHOUT_CALLBACK'] = 'BENIGN' # Replace all 'benign without callback' by 'benign'
    paths = [[prefix+(5-len(str(i)))*"0"+str(i)+".jpg"] for i in range(n_images)]
    paths = pd.DataFrame(data=paths, columns=['filename'])
    df = pd.concat([paths, descriptions[:n_images]], axis=1)
    return df

In [4]:
def extract_features(dataframe, directory, sample_count, subset):
    features = numpy.zeros(shape=(sample_count, 7, 7, 512))  # Must be equal to the output of the convolutional base
    labels = numpy.zeros(shape=(sample_count, 2))
    # Preprocess data
    generator = datagen.flow_from_dataframe(
        dataframe=dataframe, 
        directory=directory, 
        x_col="filename", y_col="pathology", class_mode="binary",
        target_size=(224,224), 
        batch_size=batch_size, shuffle=True, 
        interpolation="bilinear",
        subset=subset)
    # Pass data through convolutional base
    i = 0
    inputs_batch, labels = next(generator)
    features = conv_base.predict(inputs_batch)
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features = numpy.concatenate((features, features_batch), axis=0)
        labels = numpy.concatenate((labels, labels_batch), axis=0)
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [5]:
# Computes the total number of samples used for training phase after augmentation
def compute_num_samples(data_size, validation_ratio, aug_factor):
    train_size = int((1-validation_ratio) * data_size * aug_factor)
    return train_size

In [6]:
path_mixed = "./../CBIS-DDSM/Mix_crop_mask/CBIS-DDSM/" # Don't forget to change the paths if necessary
path_crop = "./../CBIS-DDSM/Cropped_mammo_jpg/"
path_labels = "./../CBIS-DDSM/mass_case_description_train_set.csv"
#dicom_to_jpg(path_mixed, path_crop, "MASS")
valid_ratio = 0.3
batch_size = 32
augmentation = 3
df = dataframe_generator(path_labels, 'MASS')
data_size = df.shape[0]
train_size = compute_num_samples(data_size, valid_ratio, augmentation)

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

datagen = ImageDataGenerator(
         rescale=1/255.,
         vertical_flip=True,
         horizontal_flip=True, 
         validation_split=valid_ratio)

train_features, train_labels = extract_features(df, path_crop, train_size, "training")  # Agree with our small dataset size

Instructions for updating:
Colocations handled automatically by placer.
Found 923 images belonging to 2 classes.


In [7]:
valid_size = compute_num_samples(data_size, 1-valid_ratio, 1)
validation_features, validation_labels = extract_features(df, path_crop, valid_size, "validation")  # Agree with our small dataset size

Found 395 images belonging to 2 classes.


In [ ]:
epochs = 100

model = models.Sequential()
model.add(layers.Flatten(input_shape=(7,7,512)))
model.add(layers.Dense(256, activation='relu', input_dim=(7*7*512)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))
model.summary()

# Compile model
model.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['acc'])
              
# Train model
history = model.fit(train_features, train_labels,
                    epochs=epochs,
                    batch_size=batch_size, 
                    validation_data=(validation_features, validation_labels))

In [ ]:
# Define model
from keras import models
from keras import layers
from keras import optimizers

epochs = 100

model = models.Sequential()
model.add(layers.GlobalAveragePooling2D(input_shape=(7,7,512)))
model.add(layers.Dense(2, activation='sigmoid'))
model.summary()

# Compile model
model.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['acc'])
              
# Train model
history = model.fit(train_features, train_labels,
                    epochs=epochs,
                    batch_size=batch_size, 
                    validation_data=(validation_features, validation_labels))

In [55]:
# Concatenate training and validation sets
svm_features = numpy.concatenate((train_features, validation_features))
svm_labels = numpy.concatenate((train_labels, validation_labels))

In [41]:
#svm_features[numpy.where(svm_features>1)] = 1
print(numpy.where(svm_features>1))
svm_features.shape

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


(3228, 7, 7, 512)

In [54]:
# Build model
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

X_train, y_train = svm_features.reshape(300,7*7*512), svm_labels

scaler = StandardScaler(copy=False)
scaler.fit(X_train)
scaler.transform(X_train)

param = [{
          "C": [0.01, 0.1, 1, 10, 100]
         }]
 
svm = LinearSVC(penalty='l2', loss='squared_hinge', max_iter=1000)  # As in Tang (2013)
clf = GridSearchCV(svm, param, cv=10)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [0.01, 0.1, 1, 10, 100]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)